# Step 4: Analyze results

:::{warning}

The {doc}`pycompwa </index>` is no longer maintained. Use the [ComPWA](https://compwa-org.rtfd.io) packages [QRules](https://qrules.rtfd.io), [AmpForm](https://ampform.rtfd.io), and [TensorWaves](https://tensorwaves.rtfd.io) instead!

:::

:::{note}

We load the output from the previous steps first. Note that you need to define an estimator before loading the fit result to the intensity.

:::

In [ ]:
import pycompwa.ui as pwa

In [ ]:
pwa.Logging("error")  # at this stage, we are not interested in the back-end
particle_list = pwa.read_particles("model.xml")
kinematics = pwa.create_helicity_kinematics("model.xml", particle_list)
kinematics.create_all_subsystems()
data_sample = pwa.read_root_data(input_file="generated_data.root")
phsp_sample = pwa.read_root_data(input_file="generated_phsp.root")
intensity_builder = pwa.IntensityBuilderXML(
    "model.xml", particle_list, kinematics, phsp_sample
)
fit_result = pwa.load("fit_result.xml")
intensity = intensity_builder.create_intensity()
data_set = kinematics.convert(data_sample)
phsp_set = kinematics.convert(phsp_sample)
(
    estimator,
    initial_parameters,
) = pwa.create_unbinned_log_likelihood_function_tree_estimator(
    intensity, data_set
)
intensity.updateParametersFrom(fit_result.final_parameters)

## 4.1 Convert to pandas

It is easiest to analyze your data with {class}`pandas.DataFrame`. We are mostly interested in the distributions of the kinematic variables, so we first have to convert the {class}`.DataSet` instances to a {class}`~pandas.DataFrame`. We can do this with the {mod}`.data.convert` module:

In [ ]:
from pycompwa.data import convert

In [ ]:
frame_data = convert.data_set_to_pandas(data_set)
frame_phsp = convert.data_set_to_pandas(phsp_set)
frame_data

We converted the phase space sample as well, because it serves as a uniform space over which to evaluate the intensity model. We therefore attribute an intensity as a weight to each entry in the phase space {class}`~pandas.DataFrame`:

In [ ]:
intensity_set = intensity.evaluate(phsp_set.data)
frame_phsp["weights"] = intensity_set

:::{hint}

The {mod}`pycompwa.data` contains several tools for converting from and to {class}`pandas.DataFrame`. There is also a {mod}`.data.io` submodule, which allows you to import form and export to file formats of other PWA frameworks. A demonstration of this module can be found under {doc}`/usage/tools/data_module`.

:::

## 4.2 Visualize

Now that we converted our data to {class}`pandas.DataFrame`, we can use its full potential! In this section, we will use some of that {class}`~pandas.DataFrame` functionality to create histograms of the distributions of the kinematic variables and will have a look at the Dalitz plots both data and fit result.

### Kinematic variables

Currently, the columns of the {class}`~pandas.DataFrame` created above contain final state IDs. Of course, you would rather see particle names there, but we need to use unique IDs in case there are identical particle names. The {mod}`.data.naming` module provides a tool that can convert the final state IDs to recognizable names:

In [ ]:
from pycompwa.data import naming

naming.replace_ids("2,3", "model.xml")

Now you can apply the usual {mod}`matplotlib.pyplot` tools to plot the distributions.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_1d_comparison(name, bins=100, **kwargs):
    """Helper function for comparing the 1D distributions of fit and data."""
    frame_data[name].hist(
        bins=bins, density=True, alpha=0.5, label="data", **kwargs
    )
    frame_phsp[name].hist(
        bins=bins,
        weights=frame_phsp["weights"],
        density=True,
        histtype="step",
        color="red",
        label="fit",
        **kwargs,
    )
    plt.ylabel("normalized intensity")
    title = naming.replace_ids(name, kinematics)
    plt.xlabel(title)

In [ ]:
plot_1d_comparison("theta_2_4_vs_3")
plt.legend();

Applying a transformation to one of the columns is also easily done for a {class}`pandas.DataFrame`. For instance, if you are interested in the invariant mass (and not the **square value** of the invariant mass), simply use {obj}`numpy.sqrt() <numpy.sqrt>`:

In [ ]:
from numpy import sqrt

frame_data["M(3,4)"] = sqrt(frame_data["mSq_(3,4)"])
frame_phsp["M(3,4)"] = sqrt(frame_phsp["mSq_(3,4)"])

plot_1d_comparison("M(3,4)")
plt.legend();

### Dalitz plots

Dalitz plots are 2-dimensional histograms of the square values of the invariant masses. We are therefore interested in the following variables:

In [ ]:
variable_names = data_set.data.keys()
[var for var in variable_names if var.startswith("mSq_")]

We now use {func}`~matplotlib.pyplot.hist2d` to generate the Dalitz plots:

In [ ]:
def dalitz_plot(frame, mass_x, mass_y, bins=50, **kwargs):
    """Helper function to create a Dalitz plot with useful axis titles."""
    plt.hist2d(frame[mass_x], frame[mass_y], bins=bins, **kwargs)
    plt.xlabel(naming.replace_ids(mass_x, kinematics))
    plt.ylabel(naming.replace_ids(mass_y, kinematics))

In [ ]:
from matplotlib.colors import LogNorm  # logarithmic z-axis

fig, axs = plt.subplots(1, 2, figsize=(9, 4))

plt.sca(axs[0])
axs[0].set_title("data")
dalitz_plot(frame_data, "mSq_(3,4)", "mSq_(2,4)", norm=LogNorm())

plt.sca(axs[1])
axs[1].set_title("fit")
dalitz_plot(
    frame_phsp,
    "mSq_(3,4)",
    "mSq_(2,4)",
    norm=LogNorm(),
    weights=frame_phsp["weights"],
)

## 4.3 Calculate fit fractions

The original intensity model that we optimized consists of several components. All registered component names can be retrieved via {meth}`.get_all_component_names()`:

In [ ]:
intensity_builder.get_all_component_names()

As you can see, these components are amplitudes and intensities that are added coherently or incoherently.

In PWA, you are often interested in the fractions that those amplitudes contribute to the total intensity. These **fit fractions** can be calculated using the {func}`.fit_fractions_with_propagated_errors` function. It requires amplitude or {class}`.Intensity` components that can be extracted from the {class}`.IntensityBuilderXML` instance. A nested list of the component names has to be specified as well. These are the names specified in the component XML attributes. If the inner lists contain more than one component, these components will be added coherently. In this way, you can calculate your own customized fit fractions.

In [ ]:
components = intensity_builder.create_intensity_components(
    [
        ["coherent_J/psi_-1_to_gamma_-1+pi0_0+pi0_0"],
        ["J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;"],
        ["J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;"],
        ["J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;"],
        [
            "J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;",
            "J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;",
            "J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;",
        ],
    ]
)

The last step is to call the actual fit fraction calculation function {func}`.fit_fractions_with_propagated_errors`. Here, the first required argument is a list of component pairs. Each pair resembles the nominator and denominator of a fit fraction.

In [ ]:
fit_fractions = pwa.fit_fractions_with_propagated_errors(
    [
        (components[1], components[0]),
        (components[2], components[0]),
        (components[3], components[0]),
        (components[4], components[0]),
    ],
    phsp_set,
    fit_result,
)

In [ ]:
for fraction in fit_fractions:
    print(fraction.name.replace(";", "\n  "))
    print(fraction.value, "+/-", fraction.error)
    print()

That's it! You can check some of the other {doc}`examples </usage>` to learn about more detailed features of ComPWA.

And, please, feel free to [provide feedback](https://github.com/ComPWA/pycompwa/issues/new) or [contribute](https://compwa.github.io/develop.html) ;)